In [ ]:
import pandas as pd
notes_for_sale = pd.read_csv( 'https://resources.lendingclub.com/SecondaryMarketAllNotes.csv')

In [ ]:
notes_for_sale.columns = ['LoanId', 'NoteId', 'OrderId', 'OutstandingPrincipal',
       'AccruedInterest', 'Status', 'AskPrice', 'Markup', 'YTM',
       'DaysSinceLastPayment', 'CreditScoreTrend', 'FICO',
       'DateListed', 'NeverLate', 'LoanClass', 'LoanMaturity',
       'OriginalNoteAmount', 'InterestRate', 'RemainingPayments',
       'PrincipalPlusInterest', 'ApplicationType']

notes_for_sale = notes_for_sale[notes_for_sale.AskPrice > 1]
notes_for_sale.DaysSinceLastPayment = pd.to_numeric(notes_for_sale.DaysSinceLastPayment, errors="coerce")
notes_for_sale.DateListed = pd.to_datetime(notes_for_sale.DateListed, errors="coerce", format="%m/%d/%Y")
notes_for_sale.YTM = pd.to_numeric(notes_for_sale.YTM, errors="coerce")
notes_for_sale.DaysSinceLastPayment = pd.to_numeric(notes_for_sale.DaysSinceLastPayment, errors="coerce")
notes_for_sale.CreditScoreTrend = notes_for_sale.CreditScoreTrend.astype('category')

notes_for_sale.info()

In [ ]:
notes_to_buy = notes_for_sale.query('Status == "Current" & Markup < 3 & YTM > 5 & CreditScoreTrend == "UP" & NeverLate == True & RemainingPayments <=11 &AskPrice < 20 & DaysSinceLastPayment < 31')
notes_to_buy

In [ ]:
import requests
import pandas as pd
import json

In [ ]:
api_key_info = json.load( open('C:/Users/Dharik/lc_api.json') )['ira']
investor_id = api_key_info['investor_id']
auth = api_key_info['auth']

In [ ]:
r = requests.get('https://api.lendingclub.com/api/investor/v1/accounts/' + investor_id + '/detailednotes', headers={'Authorization': auth, 'Accept':'application/json'}) 
my_notes = pd.DataFrame( r.json().get('myNotes') )
my_notes.set_index( 'loanId', inplace=True )
print( 'Got my notes: ', len(my_notes))

In [ ]:
buy_request = { "aid": investor_id, "notes": [] }
for idx, rowData in notes_to_buy.iterrows():
    LoanId = rowData['LoanId']
    if LoanId not in my_notes.index:
        buy_request["notes"].append({"loanId": LoanId, "orderId": rowData['OrderId'], "noteId":rowData['NoteId'], "bidPrice": rowData['AskPrice']} )
        
buy_request

In [ ]:
r = requests.post('https://api.lendingclub.com/api/investor/v1/accounts/' + investor_id + '/trades/buy', headers={'Authorization': auth, 'Content-Type':'application/json'}, json=buy_request) 
from pprint import pprint
pprint(r.json())